## Error Analysis: Color

Model: ViLT

In [1]:
import pandas as pd
from dataloader import COLOR_LABELS
from utils import read_json, sort_dict

In [2]:
df = pd.read_csv('../temp/vilt_col.csv')

#### Predictions

In [3]:
def _typ2cat(cols: str) -> str:
    n = len(cols.split(','))
    if n == 1:
        return 'uni'
    elif n == 2:
        return 'bi'
    elif n == 3:
        return 'tri'
    else:
        return 'many'


df['category'] = df['true'].apply(lambda x: _typ2cat(x))
df['pred'] = df['pred'].apply(lambda x: str(x))

In [4]:
label_set = list(COLOR_LABELS)

df['true_label'] = df['true'].apply(lambda s: ', '.join(label_set[int(_i)]for _i in s.split(',')))
df['pred_label'] = df['pred'].apply(lambda s: label_set[int(s)])


r-Accuracy

In [ ]:
df['correct'] = df.apply(lambda x: x['pred'] in x['true'], axis=1)

'{:.2f}'.format(df['correct'].sum() / len(df) * 100)

Confidence

In [14]:
from torch import tensor as T


def _confidence(x):
    true = x['true'].split(',')
    true = [int(_) for _ in true]

    prob = x['prob'].split(',')
    prob = T([float(_) for _ in prob])

    return prob[true].sum().item()


df['conf'] = df.apply(lambda x: _confidence(x), axis=1)

'{:.2f}'.format(df['conf'].sum() / len(df) * 100)

'43.09'

In [15]:
df.head(2)

,true,pred,prob,category,true_label,pred_label,correct,conf,obj
0,9,9,"0.05847332999110222,0.014542950317263603,0.039...",uni,gray,gray,True,0.226766,basement
1,"3,8",8,"0.04014134779572487,0.018870089203119278,0.026...",bi,"brown, white",white,True,0.569022,bathroom stall door


In [16]:
df['conf'].describe()

count    10040.000000
mean         0.430885
std          0.212883
min          0.000997
25%          0.265296
50%          0.418953
75%          0.600380
max          0.961582
Name: conf, dtype: float64

#### Object Names

In [8]:
objs = read_json('../dataset/color/test.json')
objs = list(objs)

len(df), len(objs)

(10040, 10040)

In [9]:
df['obj'] = objs

In [10]:
df.head(2)

,true,pred,prob,category,true_label,pred_label,correct,conf,obj
0,9,9,"0.05847332999110222,0.014542950317263603,0.039...",uni,gray,gray,True,0.226766,basement
1,"3,8",8,"0.04014134779572487,0.018870089203119278,0.026...",bi,"brown, white",white,True,0.569022,bathroom stall door


#### UBTM

Since our metric is *relaxed* accuracy, it's expected to see:
 #colors ↟ : score ↟

In [11]:
for cat in ['uni', 'bi', 'tri', 'many']:
    _ = df[df['category'] == cat]
    _acc = _['correct'].sum() / len(_) * 100

    print(f'{cat}: {_acc:.2f} | {len(_)}')

uni: 49.93 | 3825
bi: 65.61 | 2341
tri: 78.36 | 1146
many: 82.33 | 2728


In [13]:
for cat in ['uni', 'bi', 'tri', 'many']:
    _ = df[df['category'] == cat]
    _conf = _['conf'].sum() / len(_) * 100

    print(f'{cat}: {_conf:.2f} | {len(_)}')

uni: 26.17 | 3825
bi: 41.59 | 2341
tri: 53.89 | 1146
many: 63.56 | 2728


Object Frequency

In [17]:
csv = pd.read_csv('../results/colors.csv')

obj2freq = {o: f for o, f in zip(csv['object'], csv['total'])}

df['freq'] = df['obj'].apply(lambda o: obj2freq[o])


In [19]:
for cat in ['uni', 'bi', 'tri', 'many']:
    _ = df[df['category'] == cat]
    _freq = _['freq'].mean()

    print(f'{cat}: {_freq:.2f} | {len(_)}')

uni: 565.73 | 3825
bi: 551.12 | 2341
tri: 521.06 | 1146
many: 492.45 | 2728


In [20]:
df.head()

,true,pred,category,true_label,pred_label,correct,obj,freq
0,9,9,uni,gray,gray,True,basement,40
1,"3,8",8,bi,"brown, white",white,True,bathroom stall door,781
2,"8,9",9,bi,"white, gray",gray,True,metal drawer,55
3,"0,1,5,9,10",10,many,"red, orange, blue, gray, black",black,True,baseball gear,178
4,"3,5,8,9",3,many,"brown, blue, white, gray",brown,True,coast,202


In [22]:
obj2freq = sort_dict(obj2freq, by='v', reverse=True)

In [25]:
list(obj2freq.items())[200:300]


[('fence', 5360),
 ('sheep dog', 5351),
 ('pine tree', 5345),
 ('wall', 5341),
 ('pole street sign', 5338),
 ('camera', 5335),
 ('hotel room bed', 5307),
 ('sweater', 5282),
 ('vase flower', 5254),
 ('tree branch', 5247),
 ('hotel bed', 5193),
 ('parking lot', 5190),
 ('object', 5178),
 ('train', 5175),
 ('train station building', 5166),
 ('paint', 5158),
 ('license plate', 5135),
 ('fishing boat', 5119),
 ('chair', 5116),
 ('curtain', 5054),
 ('hat', 5033),
 ('surf board', 5029),
 ('laptop', 5008),
 ('decker train', 4991),
 ('tuxedo cat', 4975),
 ('boat', 4973),
 ('edge water', 4959),
 ('tie', 4957),
 ('freight train', 4952),
 ('book', 4947),
 ('sweatshirt', 4939),
 ('beach water', 4888),
 ('passenger train', 4880),
 ('street road', 4870),
 ('ski', 4852),
 ('racket', 4842),
 ('door car', 4830),
 ('beach umbrella', 4821),
 ('canal water', 4805),
 ('tv', 4800),
 ('skier', 4768),
 ('double-decker bus', 4757),
 ('enclosure ground', 4740),
 ('age building', 4724),
 ('surfboard riding wave'

<br>

#### ----------------------------------------------------------------------------

<br>

...